In [50]:
%matplotlib notebook
from importlib import reload  
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.random.set_seed(1)
import OUmodel
reload(OUmodel)
from mpl_toolkits import mplot3d 

In [51]:
def NN_plot(nn):
    nn.compile(loss = 'mse')
    xgrid = np.linspace(-10,10,201)
    y = nn.predict(xgrid)
    plt.plot(xgrid,y)
    plt.grid()
def plot_compare():
    NN_plot(model.Vnn)
    xgrid = np.linspace(-10,10,201)
    plt.plot(xgrid, xgrid**2)
    plt.show()
    NN_plot(model.unn)
    plt.plot(xgrid, xgrid)
    plt.show()
    NN_plot(model.dVnn)
    plt.plot(xgrid, 2*xgrid)
    plt.show()
    tmp = model.rhonn.predict(np.ones(1))[0,0]
    print('Optimal ergodic cost is: ',tmp)
    
def plotNN2d(NN):
    fig = plt.figure() 
    ax = plt.axes(projection ='3d') 
    x = y = np.arange(-3.0, 3.0, 0.05)
    X, Y = np.meshgrid(x, y)
    # z = (np.sin(X **2) + np.cos(Y **2) ) 
    XY = np.concatenate([np.reshape(X,[X.shape[0]*X.shape[1],1]),np.reshape(Y,[Y.shape[0]*Y.shape[1],1])],axis = 1)
    Z = np.reshape(NN.predict(XY),X.shape)
    ax.plot_surface(X, Y, Z) 
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    plt.show()

## Discretization

In [52]:
# Discreize parameters
steps = 20
dt = 0.01
sqrtdt = np.sqrt(dt)
T = steps * dt # 0.2
M = 10000
gamma = 0

X = np.random.normal(0,np.sqrt(1/2),size = (M,1))
Y = np.random.normal(0,0,size = (M,1))
U = Y -  X
Z = np.concatenate([U,Y],axis = 1)

# mean = [0,0]
# cov = [[1/16,1/8],[1/8,1/2]]
# Z = np.random.multivariate_normal(mean, cov, M)
# U = Z[:,:1]
# Y = Z[:,1:]

In [53]:
model = OUmodel.MODEL(steps,dt,T,M,gamma)
model.start(U, Y)
model.traindata(1)
# model.build()

### Old Method

In [5]:
# model.opt_nn.evaluate(model.x_test,model.y_test, verbose = 1)
# custom_optimizer = keras.optimizers.Adam(learning_rate = 0.01)
# model.bm_nn.compile(loss = 'mse', optimizer = custom_optimizer)
# model.bm_nn.fit(model.x_train,model.y_train,validation_data = (model.x_val,model.y_val), epochs =10, verbose = 1)
# model.bm_nn.compile(loss = 'mse', optimizer = 'Adam')
# model.bm_nn.fit(model.x_train,model.y_train,validation_data = (model.x_val,model.y_val), epochs =5, verbose = 1)
# model.rhonn.predict(np.ones(1))[0,0]
# plot_compare()

In [6]:
# print(model.unn.get_weights())
# print(model.rhonn.get_weights())
# print(model.dVnn.get_weights())
# print(model.Vnn.get_weights())

### New Method

In [58]:
# model.u_nn.set_weights([np.array([[-1],[1]], dtype='float32')])
# model.dV_nn.set_weights([np.array([[-2],[2]], dtype='float32')])
model.V_nn.set_weights([np.array([[1],[-1]], dtype='float32')])
# model.rho_nn.set_weights([np.array([[1]], dtype='float32')])

model.V_nn.predict([[10,1]]),\
model.u_nn.predict([[10,1]])

(array([[81.]], dtype=float32), array([[-0.25728285]], dtype=float32))

In [49]:
# plotNN2d(model.V_nn)
# plotNN2d(model.u_nn)

In [9]:
custom_optimizer = keras.optimizers.Adam(learning_rate = 0.01)
model.opt_nn.evaluate(model.x_train,model.y_train, verbose = 1)
model.main_nn.compile(loss = 'mse', optimizer = custom_optimizer)

10000/10000 [==============================] - 4s 440us/sample - loss: 0.0041


In [10]:
model.main_nn.fit(model.x_train,model.y_train,validation_data = (model.x_val,model.y_val), epochs =10, verbose = 1)

Train on 10000 samples, validate on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 14s 1ms/sample - loss: 0.0495 - val_loss: 0.0220
Epoch 2/10
10000/10000 [==============================] - 7s 684us/sample - loss: 0.0159 - val_loss: 0.0042
Epoch 3/10
10000/10000 [==============================] - 7s 686us/sample - loss: 0.0021 - val_loss: 0.0017
Epoch 4/10
10000/10000 [==============================] - 7s 688us/sample - loss: 0.0018 - val_loss: 0.0019
Epoch 5/10
10000/10000 [==============================] - 7s 687us/sample - loss: 0.0018 - val_loss: 0.0017
Epoch 6/10
10000/10000 [==============================] - 7s 681us/sample - loss: 0.0018 - val_loss: 0.0017
Epoch 7/10
10000/10000 [==============================] - 7s 693us/sample - loss: 0.0018 - val_loss: 0.0017
Epoch 8/10
10000/10000 [==============================] - 7s 683us/sample - loss: 0.0018 - val_loss: 0.0017
Epoch 9/10
10000/10000 [==============================] - 7s 681us/sample - loss: 0.001

In [24]:
print(model.u_nn.get_weights())
print(model.rho_nn.get_weights())
print(model.dV_nn.get_weights())
print(model.V_nn.get_weights())

[array([[ 1.],
       [-1.]], dtype=float32)]
[array([[1.]], dtype=float32)]
[array([[ 1.],
       [-1.]], dtype=float32)]
[array([[ 1.],
       [-1.]], dtype=float32)]


In [47]:
model.u_nn.set_weights([np.array([[-1],[1]], dtype='float32')])
# model.dV_nn.set_weights([np.array([[-2],[2]], dtype='float32')])
model.V_nn.set_weights([np.array([[1],[-1]], dtype='float32')])
model.rho_nn.set_weights([np.array([[1]], dtype='float32')])

In [15]:
model.main_nn.evaluate(model.x_train,model.y_train)

10000/10000 [==============================] - 3s 314us/sample - loss: 0.4334s - loss: 0.4


0.43338653557300566

In [ ]:
# model.main_nn.compile(loss = 'mse' , optimizer = 'Adam')
# model.main_nn.fit(model.x_train,model.y_train,validation_data = (model.x_val,model.y_val), epochs =100, verbose = 1)

In [ ]:
# model.main_nn.evaluate(model.x_test,model.y_test)
# model.opt_nn.evaluate(model.x_test,model.y_test)

In [ ]:
# Z_opt_end = model.opt_end_nn.predict(model.x_train)
# Z_end = model.end_nn.predict(model.x_train)
# print('start loss:',model.loss(Z).mean())
# print('optimal end loss:',model.loss(Z_opt_end).mean())
# print('numerical loss:',model.loss(Z_end).mean())
# print('theory loss:',model.rho_optimal)
# tmp = model.rho_nn.predict(np.zeros(1))[0,0]
# print('Rho_nn is: ',tmp)

u linear1d, V elu nn, dV linear1d, steps 20, samples 10000, good result

u linear1d, V elu nn + square, dV linear1d, steps 20, samples 10000, good result

u linear1d, V elu nn, dV linear1d, steps 20, samples 10000, over fitting

u linear2d, V elu nn + square 1d, dV linear2d, steps 20, samples 10000, good result

u linear2d, V linear + square 2d, dV linear2d, steps 20, samples 10000, over fitting

u linear2d, V optimal, dV linear2d, steps 20, samples 10000, good result